In [1]:
import pandas as pd
import os 
from pathlib import Path
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error 
pd.set_option('display.max_columns', 200)

In [2]:
os.getcwd()

'/Users/macintosh/github/data-science-project/predict-energy-behavior-of-prosumers'

In [3]:
INPUT_DIR = "/Users/Macintosh/Desktop/dataset/predict-energy-behavior-of-prosumers/"
# INPUT_DIR = Path("/Users/Macintosh/Downloads/predict-energy-behavior-of-prosumers/")
TEST_INPUT_DIR = "/Users/Macintosh/Desktop/dataset/predict-energy-behavior-of-prosumers/example_test_files/"


In [4]:
# Train Data
train_df = pd.read_csv(INPUT_DIR + "train.csv", sep= ",")
# train_df = pd.read_csv(INPUT_DIR / "train.csv", sep= ",")
client_df = pd.read_csv(INPUT_DIR + "client.csv", sep= ",")
electricity_prices_df = pd.read_csv(INPUT_DIR + "electricity_prices.csv", sep= ",")
forecast_weather_df = pd.read_csv(INPUT_DIR + "forecast_weather.csv", sep= ",")
gas_prices_df = pd.read_csv(INPUT_DIR + "gas_prices.csv", sep= ",")
historical_weather_df = pd.read_csv(INPUT_DIR + "historical_weather.csv", sep= ",")
weather_station_to_county_mapping_df = pd.read_csv(INPUT_DIR + "weather_station_to_county_mapping.csv", sep= ",")

In [5]:
# Test Data
test_df = pd.read_csv(TEST_INPUT_DIR + "test.csv")
sample_submission_df = pd.read_csv(TEST_INPUT_DIR + "sample_submission.csv")

In [6]:
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression

In [7]:
# Create data frame for predict value
# ALL_IDS = clean_df.row_id.unique()
# oof = pd.DataFrame(data=np.zeros((len(ALL_IDS), 1)), index=ALL_IDS, columns=['target'])
oof = train_df.loc[:, ['row_id', 'data_block_id', 'target']]
oof = oof.set_index('row_id')
oof['target'] = 0

In [8]:
oof.head()

,data_block_id,target
row_id,,
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [9]:
# FEATURES = [i for i in train_df.select_dtypes(include='number').columns if i != 'target']

In [15]:
train_df.select_dtypes(include='number').drop(columns=[''])

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,0,0,0
1,0,0,1,96.590,1,0,1,0
2,0,0,2,0.000,0,0,2,1
3,0,0,2,17.314,1,0,3,1
4,0,0,3,2.904,0,0,4,2


In [16]:
FEATURES = ['county', 'is_business', 'product_type', 'is_consumption', 'prediction_unit_id']

In [17]:
model = {}

In [18]:
# clean_df = train_df[~train_df['target'].isna()].reset_index(drop=True)
clean_df = train_df.fillna(value={'target':-999})

In [19]:
clean_df.head()

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
0,0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0
1,0,0,1,96.590,1,2021-09-01 00:00:00,0,1,0
2,0,0,2,0.000,0,2021-09-01 00:00:00,0,2,1
3,0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1
4,0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2


In [20]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2018352 entries, 0 to 2018351
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   county              int64  
 1   is_business         int64  
 2   product_type        int64  
 3   target              float64
 4   is_consumption      int64  
 5   datetime            object 
 6   data_block_id       int64  
 7   row_id              int64  
 8   prediction_unit_id  int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 138.6+ MB


In [56]:
def linear_predict(model_dict, model_name, data, output_data, features):
    gkf = GroupKFold(n_splits=5)
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=data, groups=data.index)):
        print(f'Fold {i+1}')
        print('=======')
        train_x = data.loc[train_idx]
        train_y = data.loc[train_idx]
        valid = data.loc[valid_idx]
        valid_ids = valid.row_id.values

        reg = LinearRegression()
        reg.fit(train_x[features], train_y['target'])

        model_dict[f'{model_name}'] = reg 
        output_data.loc[valid_ids,['target']] = reg.predict(valid[features])
    return output_data

In [57]:
import xgboost as xgb 

In [58]:
def xgb_predict(model_dict, model_name, data, output_data, features):
    gkf = GroupKFold(n_splits=5)
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=data, groups=data.index)):
        print(f'Fold {i+1}')
        print('=======')
        train_x = data.loc[train_idx]
        train_y = data.loc[train_idx]
        valid = data.loc[valid_idx]
        valid_ids = valid.row_id.values

        xgb_r = xgb.XGBRegressor(objective='reg:linear', n_estimators=10, seed=123)
        xgb_r.fit(train_x[features], train_y['target'])

        model_dict[f'{model_name}'] = xgb_r
        output_data.loc[valid_ids, ['target']] = xgb_r.predict(valid[features])
    return output_data


In [59]:
from lightgbm import LGBMRegressor

In [64]:
def lgbm_predict(model_dict, model_name, data, output_data, features):
    gkf = GroupKFold(n_splits=5)
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=data, groups=data.index)):
        print(f'KFold : {i+1}')
        print('=======')
        train_x = data.loc[train_idx]
        train_y = data.loc[train_idx]
        valid = data.loc[valid_idx]
        valid_ids = valid.row_id.values

        lgbm_r = LGBMRegressor(n_estimators=10, seed=123)
        lgbm_r.fit(train_x[features], train_y['target'])

        model_dict[f'{model_name}'] = lgbm_r 
        output_data.loc[valid_ids, ['target']] = lgbm_r.predict(valid[features])
    return output_data

In [69]:
from sklearn.neighbors import KNeighborsRegressor

In [70]:
def knb_predict(model_dict, model_name, data, output_data, features):
    gkf = GroupKFold(n_splits=5)
    for i, (train_idx, valid_idx) in enumerate(gkf.split(X=data, groups=data.index)):
        print(f'KFold : {i+1}')
        print('=======')
        train_x = data.loc[train_idx]
        train_y = data.loc[train_idx]
        valid = data.loc[valid_idx]
        valid_ids = valid.row_id.values

        knb = KNeighborsRegressor(n_neighbors=3)
        knb.fit(train_x[features], train_y['target'])

        model_dict[f'{model_name}'] = knb 
        output_data.loc[valid_ids, ['target']] = knb.predict(valid[features])
    return output_data

In [22]:
# oof.shape

(2018352, 2)

In [71]:
# oof = linear_predict(model, 'first', clean_df, oof, FEATURES)
# oof = xgb_predict(model, 'xgb', clean_df, oof, FEATURES)
# oof = lgbm_predict(model, 'lgbm', clean_df, oof, FEATURES)
oof = knb_predict(model, 'knb', clean_df, oof, FEATURES)

KFold : 1
KFold : 2
KFold : 3
KFold : 4
KFold : 5


In [72]:
def mae(df_pred, df_train):
    return mean_absolute_error(oof['target'], clean_df['target'])

In [73]:
mae(oof, clean_df)

160.72146725827145

In [635]:
oof.shape[0]

2018352